# FuseTS - MOGPR S1 S2 / Phenology openEO Workflow

In this notebook, we'll demonstrate how to use openEO's diverse capabilities to build a workflow for fusing data and executing time series analysis. In this particular case, we'll integrate both FuseTS's MOGPR service and the Phenology service into a single workflow to compute phenology metrics for a specific field. The end product of this process will be a NetCDF file containing the final phenology metrics.

**Prerequisites**
- In this notebook, we utilize openEO to retrieve time series data. To leverage the full range of openEO features, you can create a free trial account on the [openEO Platform](https://docs.openeo.cloud/join/free_trial.html) and receive 1000 free credits, enabling you to execute the functionalities showcased in this notebook.


Lets start with importing the different libraries that we need within this notebook.


## Setting up the environment

First of all we need to make sure that all our dependencies are correctly installed.


In [12]:
!pip install openeo

In [13]:

import json
import warnings

import numpy as np
import openeo
from openeo.metadata import CollectionMetadata, BandDimension, Band
from ipyleaflet import GeoJSON, Map, basemaps

warnings.filterwarnings("ignore")

# Exploring the area of interest
In this first part, we will explore the area of interest that will be used for calculating the phenology metrics.

In [14]:
year = 2023

spat_ext = {
        "type": "Polygon",
        "coordinates": [
            [
                [5.170012098271149, 51.25062964728295],
                [5.17085904378298, 51.24882567194015],
                [5.17857421368097, 51.2468515482926],
                [5.178972704726344, 51.24982704376254],
                [5.170012098271149, 51.25062964728295],
            ]
        ],
    }
temp_ext = [f"{year}-01-01", f"{year}-12-30"]

Plot the area to see what we're working with.


In [15]:
center = np.mean(spat_ext["coordinates"][0], axis=0).tolist()[::-1]
zoom = 16

m = Map(basemap=basemaps.Esri.WorldImagery, center=center, zoom=zoom)
g = GeoJSON(
    data=spat_ext,
    style={
        "color": "red",
        "opacity": 1,
        "weight": 1.9,
        "dashArray": "9",
        "fillOpacity": 0.5,
    },
)
m.add(g)
m

Map(center=[51.249352711712234, 5.173686031746518], controls=(ZoomControl(options=['position', 'zoom_in_text',…

# Setting up connection with openEO

The first step in setting up the workflow, is creating a connection with openEO. 

In [16]:
connection = openeo.connect("openeo.vito.be").authenticate_oidc()

Authenticated using refresh token.


# Creating the fused data set using MOGPR

In the following section, we will leverage the capabilities of openEO to generate a fused dataset using MOGPR.

In [17]:
service = 'mogpr_s1_s2'
namespace = 'u:fusets'
mogpr = connection.datacube_from_process(service,
                                         namespace=f'https://openeo.vito.be/openeo/1.1/processes/{namespace}/{service}',
                                         polygon=spat_ext, date=temp_ext, s1_collection='RVI ASC', s2_collection='NDVI')

mogpr.metadata = CollectionMetadata(metadata={},dimensions=[BandDimension(name='bands',bands=[Band('RVI ASC'),Band('NDVI')])])

# Adding phenology service

After setting up the base, fused datacube through MOGPR, we can extend the workflow with the phenology calculation.

In [18]:
mogpr_ndvi = mogpr.band('NDVI')

In [19]:
service = 'phenology'
namespace = 'u:fusets'
phenology = connection.datacube_from_process(service,
                                             namespace=f'https://openeo.vito.be/openeo/1.1/processes/{namespace}/{service}',
                                             data=mogpr_ndvi)

# Downloading the results
Now that we've setup our workflow, we can now download the results by starting an openEO batch job.

In [20]:
phenology_output_file = './phenology.nc'
phenology_job = phenology.execute_batch(phenology_output_file, out_format="netcdf",
                                        title=f'FuseTS - MOGPR - Phenology', job_options={
        'udf-dependency-archives': [
            'https://artifactory.vgt.vito.be:443/artifactory/auxdata-public/ai4food/fusets_venv.zip#tmp/venv',
            'https://artifactory.vgt.vito.be:443/artifactory/auxdata-public/ai4food/fusets.zip#tmp/venv_static'
        ]
    })

0:00:00 Job 'j-2403265ce42148bea231601c154187d3': send 'start'
0:00:27 Job 'j-2403265ce42148bea231601c154187d3': queued (progress N/A)
0:00:32 Job 'j-2403265ce42148bea231601c154187d3': queued (progress N/A)
0:00:38 Job 'j-2403265ce42148bea231601c154187d3': queued (progress N/A)
0:00:46 Job 'j-2403265ce42148bea231601c154187d3': queued (progress N/A)
0:00:56 Job 'j-2403265ce42148bea231601c154187d3': queued (progress N/A)
0:01:09 Job 'j-2403265ce42148bea231601c154187d3': queued (progress N/A)
0:01:25 Job 'j-2403265ce42148bea231601c154187d3': queued (progress N/A)
0:01:44 Job 'j-2403265ce42148bea231601c154187d3': queued (progress N/A)
0:02:08 Job 'j-2403265ce42148bea231601c154187d3': queued (progress N/A)
0:02:39 Job 'j-2403265ce42148bea231601c154187d3': queued (progress N/A)
0:03:16 Job 'j-2403265ce42148bea231601c154187d3': running (progress N/A)
0:04:04 Job 'j-2403265ce42148bea231601c154187d3': running (progress N/A)
0:05:03 Job 'j-2403265ce42148bea231601c154187d3': running (progress N/A

# Explore the results¶

In [21]:
import netCDF4 as nc
import matplotlib.pyplot as plt
import math

nc_file = nc.Dataset(phenology_output_file)
keys = [x for x in nc_file.variables.keys() if x not in ['phenology', 'x', 'y', 'crs']]
keys.sort()

lat = nc_file.variables['x'][:]
lon = nc_file.variables['y'][:]

ncols = 2
nrows = math.ceil(len(keys) / ncols)
col = 0
row = 0

fig, axs = plt.subplots(nrows, ncols, figsize=(15, 30))

for i, key in enumerate(keys):

    variable = nc_file.variables[key][0]
    im_pred = axs[row, col].imshow(variable)
    axs[row, col].set_title(key)
    fig.colorbar(im_pred, ax=axs[row, col])

    col = (col + 1) % ncols
    if col == 0:
        row += 1
plt.show()

ModuleNotFoundError: No module named 'netCDF4'